In [60]:
import pandas as pd 
import git
import numpy as np
import geopandas as gpd
import plotly as px
import networkx as nx
from datetime import datetime as dt
from vehicle_stream_pipeline.utils import get_shortest_ride, get_hotspots, calculate_drives, calculate_graph, add_drone_flights, poi_poly

repo = git.Repo(".", search_parent_directories=True).git.rev_parse(
    "--show-toplevel")

In [61]:
drives = pd.read_csv(f"{repo}/data/cleaning/data_cleaned.csv")
drives = drives[(drives["state"] == "completed")]
edges = pd.read_excel(f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name= 'Liste 2022')

/var/folders/gb/hn_j3jkn5_5fm5ylgtcd9w_00000gn/T/ipykernel_13654/3440087016.py:1: DtypeWarning: Columns (44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  drives = pd.read_csv(f"{repo}/data/cleaning/data_cleaned.csv")


KeyboardInterrupt: 

In [71]:
edges

,start_id,Start Name,end_id,Ende Name,Route [m],Luftlinie [m],VRN-eTarif\nohne Bahncard,VRN-eTarif\nmit BahnCard,Qualitätszuschlag,MoD-Fahrpreis \nohne BahnCard,MoD-Fahrpreis\n mit Bahncard,0.12680550240634963,0.07159177456207169,0.18042226487523982,0.5159630940478845,0.2894135567402895,0.72936660268714,price_mod,price_mod_2pers,price_mod_3pers,price_mod_4pers,price_bahncard,price_bahncard_2pers,price_bahncard_3pers,price_bahncard_4pers,price_vrn_surcharge,price_vrn_surcharge_2pers,price_vrn_surcharge_3pers,price_vrn_surcharge_4pers
0,1001,Mandelgasse,1002,Hauptfeuerwache,366,285,1.65,1.24,0.900,2.550,2.140,0.160784,NaN,NaN,0.647059,NaN,NaN,2.550,4.275,6.000,7.725,2.140,3.865,5.180,6.700,0.900,2.625,3.940,5.460
1,1001,Mandelgasse,1003,Kindergarten St. Marien,994,586,1.65,1.24,0.900,2.550,2.140,0.160784,NaN,NaN,0.647059,NaN,NaN,2.550,4.275,6.000,7.725,2.140,3.865,5.180,6.700,0.900,2.625,3.940,5.460
2,1001,Mandelgasse,1004,Strohmarkt,796,694,1.65,1.24,0.900,2.550,2.140,0.160784,NaN,NaN,0.647059,NaN,NaN,2.550,4.275,6.000,7.725,2.140,3.865,5.180,6.700,0.900,2.625,3.940,5.460
3,1001,Mandelgasse,1005,Rathaus,555,510,1.65,1.24,0.900,2.550,2.140,0.160784,NaN,NaN,0.647059,NaN,NaN,2.550,4.275,6.000,7.725,2.140,3.865,5.180,6.700,0.900,2.625,3.940,5.460
4,1001,Mandelgasse,1006,Villenstraße,1416,947,1.65,1.24,1.175,2.825,2.415,0.145133,NaN,NaN,0.584071,NaN,NaN,2.825,4.825,6.825,8.825,2.415,4.415,6.005,7.800,1.175,3.175,4.765,6.560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40195,18005,Festhalle Geinsheim,15014,Loblocher Straße,14100,10281,4.15,3.14,5.605,9.755,8.745,0.103537,NaN,NaN,0.425423,NaN,NaN,9.755,17.435,25.115,32.795,8.745,16.425,23.095,30.270,5.605,13.285,19.955,27.130
40196,18005,Festhalle Geinsheim,15015,Am Krautgarten,12650,8787,3.65,2.76,5.335,8.985,8.095,0.099054,NaN,NaN,0.406233,NaN,NaN,8.985,16.145,23.305,30.465,8.095,15.255,21.525,28.240,5.335,12.495,18.765,25.480
40197,18005,Festhalle Geinsheim,16009,In den Obergärten,6933,5236,2.90,2.19,2.950,5.850,5.140,0.121368,NaN,NaN,0.495726,NaN,NaN,5.850,10.250,14.650,19.050,5.140,9.540,13.230,17.275,2.950,7.350,11.040,15.085
40198,18005,Festhalle Geinsheim,16027,Altes Schulhaus Lachen,5694,4883,2.65,2.00,2.485,5.135,4.485,0.126582,NaN,NaN,0.516066,NaN,NaN,5.135,8.945,12.755,16.565,4.485,8.295,11.455,14.940,2.485,6.295,9.455,12.940


In [ ]:
df_stops = pd.read_excel(
    f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name='MoDstops')

In [ ]:
startdate = drives.scheduled_to.min()
startdate = dt.strptime(startdate, '%Y-%m-%d %H:%M:%S')
startdate = startdate.strftime('%m/%d/%Y')
start_date = dt.strptime(startdate, "%m/%d/%Y")
enddate = drives.scheduled_to.max()
enddate = dt.strptime(enddate, '%Y-%m-%d %H:%M:%S')
enddate = enddate.strftime('%m/%d/%Y')
end_date = dt.strptime(enddate, "%m/%d/%Y")

In [ ]:
aggregated_drives = calculate_drives(drives, start_date, end_date)

In [62]:
graph = calculate_graph(aggregated_drives)

In [7]:
hotspots = get_hotspots(edges, aggregated_drives)

/usr/local/lib/python3.9/site-packages/vehicle_stream_pipeline/utils.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_edges_filtered["Spots"] = df_edges_filtered.apply(


In [8]:
hotspots = [spot[0] for spot in hotspots]

In [9]:
hotspots

[1008, 4025, 1005, 1009, 1007, 12007, 7001, 6004, 1010, 11017]

In [66]:
hotspots = [1008, 4025, 1005, 1009, 1007, 12007, 7001, 6004, 1010, 11017]
drone_spots = [15011, 13001, 2002, 11007, 4016, 1002, 3020, 9019, 9005]

In [136]:
def add_drone_flights(df_edges, drives, drone_spots=[1008], radius=500):
    drone_flights = df_edges.iloc[:, :6]
    drone_flights.drop(
        ["Start Name", "Ende Name", "Route [m]"], axis=1, inplace=True)
    drone_flights.rename(columns={"Luftlinie [m]": "Luftlinie"}, inplace=True)

    drone_flights = drone_flights[drone_flights["Luftlinie"] <= radius]

    drone_flights["number_of_drives"] = 1
    drone_flights["waiting_time"] = 0
    drone_flights["avg_ride_time"] = (
        (drone_flights.Luftlinie * 0.12) / 60 / 60 / 24
    )
    drone_flights["avg_time_to_destination"] = (
        (drone_flights.Luftlinie * 0.12) / 60 / 60 / 24
    )

    drone_flights = drone_flights[
        (drone_flights["start_id"].isin(drone_spots))
        | (drone_flights["end_id"].isin(drone_spots))
    ]

    
    drone_flights.rename(columns={"start_id": "pickup_address", "end_id": "dropoff_address"}, inplace=True)
    drone_flights.drop(["Luftlinie"], axis=1, inplace=True)

    drives_w_flights = pd.merge(
        drives,
        drone_flights,
        left_on=["pickup_address", "dropoff_address"],
        right_on=["pickup_address", "dropoff_address"],
        how="left",
    )

    drives_w_flights["start_end"] = drives_w_flights["pickup_address"]  + drives_w_flights["dropoff_address"]
    drone_flights["start_end"] = drone_flights["pickup_address"]  + drone_flights["dropoff_address"]

    drone_flights_added = drone_flights[~(drone_flights["start_end"].isin(drives_w_flights["start_end"]))]

    drives_w_flights["avg_time_to_destination"] = np.where(
        (
            drives_w_flights["avg_time_to_destination_x"]
            > drives_w_flights["avg_time_to_destination_y"]
        ),
        drives_w_flights.avg_time_to_destination_y,
        drives_w_flights.avg_time_to_destination_x,
    )

    drives_w_flights = drives_w_flights[
        [
            "pickup_address",
            "dropoff_address",
            "avg_time_to_destination",
        ]
    ]

    drone_flights_added = drone_flights_added[
        [
            "pickup_address",
            "dropoff_address",
            "avg_time_to_destination",
        ]
    ]


    drives_w_flights = pd.concat([drives_w_flights, drone_flights_added])

    return drives_w_flights


In [137]:
edges.rename(columns={"Start #": "start_id",
                "Ende #": "end_id"}, inplace=True)
drives_w_flights = add_drone_flights(edges, aggregated_drives, drone_spots = drone_spots, radius = 500)

Index(['pickup_address', 'dropoff_address', 'number_of_drives', 'waiting_time',
       'avg_ride_time', 'avg_time_to_destination', 'start_end'],
      dtype='object')


In [139]:
print(drives_w_flights[(drives_w_flights["pickup_address"] == 1002) & (drives_w_flights["dropoff_address"] == 1001)])


     pickup_address  dropoff_address  avg_time_to_destination
200            1002             1001                 0.000396


In [9]:
graph_with_drones = calculate_graph(drives_w_flights)

In [10]:
result1, a = get_shortest_ride(1001, 10001, graph)
result2, b = get_shortest_ride(1001, 10001, graph_with_drones)

In [24]:
result1

[1001, 1005, 4025, 1008, 4038, 10001]

In [37]:
result2

([1001, 1005, 1009, 1008, 4038, 10001], 290.3547233796296)

In [76]:
def find_id_for_name(name, df_stops):
    return df_stops[df_stops["MoDStop Name"] == name]["MoDStop Id"].reset_index(drop=True)[0]

In [87]:
find_id_for_name("Rathaus", df_stops)

1005